In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from io import StringIO
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from IPython.display import clear_output
from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Electrics
import keyboard
import time

def millis():
    return time.time_ns() / 1000000
oldtime = millis()
curtime = 0
rectime = 0
step = 1/60 * 1000 #60 hz

## Lectura de datos

In [2]:
# Leer carpeta
CARPETA = "datos_beamng"
datos = False
columnas = [
    # Comandos
    "throttle",
    "brake",
    "clutch",
    "gear",
    # Estado
    "throttle_input",
    "brake_input",
    "rpm",
    "airspeed"
]
files = Path(CARPETA).glob('*')
for i, ruta in enumerate(files):
    df = pd.read_csv(ruta, usecols=columnas + ["is_shifting"])
    if (i == 0):
        datos = df
    else:
        datos = pd.concat([datos, df], ignore_index=True)
# MANTENER CAMBIO
gear = 0
for i in range(datos.shape[0]):
    if (datos["is_shifting"].iloc[i] == True):
        if (datos["gear"].iloc[i] == 0):
            datos.at[i, "gear"] = gear
    else:
        gear = datos["gear"].iloc[i]
datos.drop(columns="is_shifting", inplace=True)
datos = datos[columnas]
display(datos)

,throttle,brake,clutch,gear,throttle_input,brake_input,rpm,airspeed
0,0.0,0.3,1.0,1.0,0.0,0.0,677.650753,0.000130
1,0.0,0.3,1.0,1.0,0.0,0.0,674.418942,0.000126
2,0.0,0.3,1.0,1.0,0.0,0.0,677.589228,0.000147
3,0.0,0.3,1.0,1.0,0.0,0.0,680.929954,0.000171
4,0.0,0.3,1.0,1.0,0.0,0.0,677.760520,0.000180
...,...,...,...,...,...,...,...,...
26243,0.0,0.3,1.0,1.0,0.0,0.0,674.727744,0.011388
26244,0.0,0.3,1.0,1.0,0.0,0.0,677.971016,0.009939
26245,0.0,0.3,1.0,1.0,0.0,0.0,681.158786,0.009315
26246,0.0,0.3,1.0,1.0,0.0,0.0,678.049100,0.009805


## Normalizar

In [3]:
scaler = MaxAbsScaler().fit(datos)
datos_scaled = pd.DataFrame(scaler.transform(datos), index=datos.index, columns=datos.columns)

## Entrada/Salida y perceptrón multicapa

In [4]:
inp = ["rpm", "airspeed", "throttle_input", "brake_input"]
out = ["throttle", "brake", "clutch", "gear", ]

X = datos_scaled[inp]
Y = datos_scaled[out]
mlp = MLPRegressor(
    hidden_layer_sizes=(256, 256, 256, 256, 256, 256),
    verbose=True)
mlp.fit(X, Y)

Iteration 1, loss = 0.02094680
Iteration 2, loss = 0.01529557
Iteration 3, loss = 0.01470803
Iteration 4, loss = 0.01440890
Iteration 5, loss = 0.01409677
Iteration 6, loss = 0.01383268
Iteration 7, loss = 0.01365778
Iteration 8, loss = 0.01348983
Iteration 9, loss = 0.01328512
Iteration 10, loss = 0.01308330
Iteration 11, loss = 0.01271826
Iteration 12, loss = 0.01255753
Iteration 13, loss = 0.01225385
Iteration 14, loss = 0.01196533
Iteration 15, loss = 0.01182142
Iteration 16, loss = 0.01153632
Iteration 17, loss = 0.01148571
Iteration 18, loss = 0.01137565
Iteration 19, loss = 0.01111706
Iteration 20, loss = 0.01090185
Iteration 21, loss = 0.01116427
Iteration 22, loss = 0.01067649
Iteration 23, loss = 0.01042254
Iteration 24, loss = 0.01061244
Iteration 25, loss = 0.01010329
Iteration 26, loss = 0.01005747
Iteration 27, loss = 0.01040174
Iteration 28, loss = 0.00973634
Iteration 29, loss = 0.00987345
Iteration 30, loss = 0.00978162
Iteration 31, loss = 0.00953814
Iteration 32, los

MLPRegressor(hidden_layer_sizes=(256, 256, 256, 256, 256, 256), verbose=True)

## Verificación

In [5]:
bng_path = "C:\Program Files (x86)\Steam\steamapps\common\BeamNG.drive"
bng = BeamNGpy('localhost', 64256, home=bng_path)
bng.open()

In [6]:
scenario = Scenario('west_coast_usa', 'ml_beamng')
vehicle = Vehicle('veh', model='pessima', licence='HOLA')
electrics = Electrics()
vehicle.attach_sensor("electrics", electrics)
scenario.add_vehicle(vehicle, pos=(-717, 101, 118), rot_quat=(0, 0, 0.3826834, 0.9238795))
scenario.make(bng)
bng.load_scenario(scenario)
bng.start_scenario()
vehicle.set_shift_mode("realistic_manual")

In [ ]:
oldspeed = 0
oldgear = 0
def lectura_sensores():
    global oldspeed
    global oldgear
    # LECTURA
    vehicle.poll_sensors()
    electrics.data.pop("wheelThermals")            
    x = pd.DataFrame(electrics.data.values(), index=electrics.data.keys()).transpose()
    x = x[columnas]
    # MANTENER CAMBIO
    if electrics.data["is_shifting"]:
        if electrics.data["gear"] == 0:
            x.at[0, "gear"] = oldgear
    else:
        oldgear = electrics.data["gear"]
    return x

oldtime = 0
while True:
    cmd1 = 0
    cmd2 = 0
    if (keyboard.is_pressed('D')):
        break
    elif (keyboard.is_pressed('W')):
        cmd1 = 0.7
        cmd2 = 0
    elif (keyboard.is_pressed('S')):
        cmd1 = 0
        cmd2 = 0.5
    #actualizar valor
    curtime = millis()
    if curtime > (oldtime + step):
        oldtime = curtime
        x = lectura_sensores()
        x["throttle_input"] = cmd1
        x["brake_input"] = cmd2
        x_scaled = pd.DataFrame(scaler.transform(x), index=x.index, columns=x.columns)
        output = dict(zip(out, mlp.predict(x_scaled[inp]).tolist()[0]))
        vehicle.control(
            throttle=output["throttle"],
            brake=output["brake"],
            clutch=output["clutch"],
            gear=round(output["gear"]*5)
        )
        display(output)
        clear_output(wait = True)

{'throttle': 0.35124755162588656,
 'brake': 0.006242211517956953,
 'clutch': 0.37474610084180693,
 'gear': 0.15613829520873826}